### Generate an environment
Currently, this just copies data/default_env.pickle into data/env.pickle.

**To do**: Add a DVC parameter such as `env_name` and if this isn't set to "default" then generate a new environment.

In [ ]:
!dvc run --force --always-changed -n gen_env -w .. -d data/default_env.pickle -o data/env.pickle python scripts/gen_env.py

### Generate observations
Currently this checks the values of the parameters `target_area`, `true_norm`, `num_observations` and `obs_data_set`. If they are equal to the settings of the observations in data/default_observations.pickle (and `obs_data_set` has value 1), then that file is copied into data/observations.pickle.

**To do**: Generate a new set of observations if the parameter settings don't match the default one, or `obs_data_set` is not 1.


In [1]:
!dvc run --force --always-changed -n gen_observations -w .. -d data/env.pickle -p target_area,true_norm,num_observations,obs_data_set -o data/observations.pickle python scripts/gen_observations.py

Running stage 'gen_observations' with command:
	python scripts/gen_observations.py
(-0.8, 0.7)
(0.25, 0.99)
[3, 8, 13, 35, 40]
Modifying stage 'gen_observations' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml

Repetition of Task: 100%|██████████| 50/50 [00:00<00:00, 752.75it/s]


### Check log likelihood of observation set given no norm and true norm

In [3]:
!dvc run --force --always-changed -n test_obs_log_likelihood1 -w .. -d data/observations.pickle -d data/env.pickle -p true_norm.exp python scripts/test_obs_likelihood.py none true

Running stage 'test_obs_log_likelihood1' with command:
	python scripts/test_obs_likelihood.py none true
Log likelihood of observations given no norm: -13732.653608350562

Log likelihood of observations given true exp.: -12694.183534230084

Modifying stage 'test_obs_log_likelihood1' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock


### Generate MCMC chains
Creates data/chains_and_log_posteriors.pickle. Also metrics/starts_info.txt, but that is not currently recorded in the "dvc run" command below.

In [3]:
!dvc run --force -n gen_mcmc_chains -w .. -d data/env.pickle -d data/observations.pickle -p n,m,rf,colour_specific,shape_specific,target_area -o data/chains_and_log_posteriors.pickle python scripts/gen_mcmc_chains.py

Running stage &#39;gen_mcmc_chains&#39; with command:Traceback (most recent call last):
  File &quot;scripts/gen_mcmc_chains.py&quot;, line 60, in &lt;module&gt;
    pickle_it(dask.compute(*chains_and_log_posteriors), &#39;data/chains_and_log_posteriors.pickle&#39;)
  File &quot;C:\Users\crast78p\OneDrive - University of Otago\PythonProjects\my fork - learning_norms_with_mcmc_from_pcfg\env\lib\site-packages\dask\base.py&quot;, line 452, in compute
    results = schedule(dsk, keys, **kwargs)
  File &quot;C:\Users\crast78p\OneDrive - University of Otago\PythonProjects\my fork - learning_norms_with_mcmc_from_pcfg\env\lib\site-packages\dask\threaded.py&quot;, line 76, in get
    results = get_async(
  File &quot;C:\Users\crast78p\OneDrive - University of Otago\PythonProjects\my fork - learning_norms_with_mcmc_from_pcfg\env\lib\site-packages\dask\local.py&quot;, line 486, in get_async
    raise_exception(exc, tb)
  File &quot;C:\Users\crast78p\OneDrive - University of Otago\PythonProjects\m

### Analyse the chains
Generate file metrics/chain_posteriors.csv and append more information to metrics/chain_info.txt (note: DVC may not handle changing a file very well - perhaps a separate file should be created).

In [1]:
!dvc run --force -n analyse_chains -w .. -d data/chains_and_log_posteriors.pickle -d data/env.pickle -d data/observations.pickle -p n,m,rf,colour_specific,shape_specific,target_area -o metrics/chain_info.txt --plots metrics/chain_posteriors.csv python scripts/analyse_chains.py

17133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13701.015867017133, -13702.114479305801, -13702.114479305801, -13702.114479305801, -13702.114479305801, -13702.114479305801, -13702.114479305801, -13702.114479305801, -13702.114479305801, -13702.114479305801, -13733.346755531122, -13733.346755531122, -13733.346755531122, -13733.346755531122, -13733.346755531122, -13733.346755531122, -13733.346755531122, -13733.346755531122, -13733

### Generate a plot showing norms ordered by likelihood (or now posterior prob.?)
The file plots.html is generated in the root folder, based on data in metrics/chain_posteriors.csv

In [4]:
!pushd .. && pwd && dvc plots show -t plots/norm_exp_histogram.json metrics/chain_posteriors.csv

/c/Users/crast78p/OneDrive - University of Otago/PythonProjects/my fork - learning_norms_with_mcmc_from_pcfg

ERROR: unexpected error - 'metrics/chain_posteriors.csv'
Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!
DVC version: 1.8.1 (pip)
---------------------------------
Platform: Python 3.8.2 on Windows-10-10.0.18362-SP0
Supports: gdrive, http, https
Cache types: hardlink
Cache directory: NTFS on C:\
Workspace directory: NTFS on C:\
Repo: dvc, git


### Perform a convergence test to compute $\hat{R}$
The result is written to metrics/conv_test.txt. Also, the posterior sample (after discarding initial warm-up segments of chains and combining the remains) is writen to data/posterior.pickle. 


In [2]:
!dvc run --force -n conv_test -w .. -d data/chains_and_log_posteriors.pickle -p rhat_step_size -o data/posterior.pickle -o metrics/conv_test.txt python scripts/conv_test.py

Running stage 'conv_test' with command:
	python scripts/conv_test.py

Calculating Variance for 1st 50 iterations
Temp. check: seq length is 50
Time taken for job=0.4s

Calculating Variance for 1st 100 iterations
Temp. check: seq length is 100
Time taken for job=0.7s

Calculating Variance for 1st 200 iterations
Temp. check: seq length is 200
Time taken for job=1.3s

Calculating Variance for 1st 400 iterations
Temp. check: seq length is 400
Time taken for job=2.7s

Calculating Variance for 1st 800 iterations
Temp. check: seq length is 800
Time taken for job=5.9s

Calculating Variance for 1st 1600 iterations
Temp. check: seq length is 1600
Time taken for job=10.5s

Calculating Variance for 1st 3200 iterations
Temp. check: seq length is 3200
Time taken for job=20.5s

Calculating Variance for 1st 4999 iterations
Temp. check: seq length is 4999
Time taken for job=30.9s

Modifying stage 'conv_test' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.

### Extract the top norms and compute precision and recall
The results are written to metrics/precision_recall.txt.


In [3]:
!dvc run --force --always-changed -n extract_top_norms -w .. -d data/posterior.pickle -d data/env.pickle -d metrics/chain_posteriors.csv -p colour_specific,shape_specific,target_area,true_norm -o data/top_norms.pickle -o metrics/precision_recall.txt python scripts/extract_top_norms.py

Running stage 'extract_top_norms' with command:
	python scripts/extract_top_norms.py
True expression passed to performance(...): ['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]
true_data[0]: ((('pickup', 3), ('putdown', 3, '1')), (('pickup', 40), ('putdown', 40, '3')), (('pickup', 13), ('putdown', 13, '3')), (('pickup', 8), ('putdown', 8, '3')), (('pickup', 35), ('putdown', 35, '1')))
Calculting P&R for exp (('Norms', ('Obl', ('Moved', ('Colour', 'r'), ('Shape', 'any'), ('Zone', '1'), ('Next-Move', ('Colour', 'any'), ('Shape', 'any'))), ('Zone', '2')), ('Per', ('Action', 'putdown'), ('Colour', 'any'), ('Shape', 'square'), ('PerZone', '3'))), 13322) at rank 1
predicted_data[0]: ((('pickup', 35), ('putdown', 35, '1')), (('pickup', 3), ('putdown', 3, '2')), (('pickup', 40), ('putdown', 40, '3')), (('pic

### Check log likelihood of observation set given top norm

In [4]:
!dvc run --force --always-changed -n test_obs_log_likelihood2 -w .. -d data/observations.pickle -d data/env.pickle -d data/top_norms.pickle python scripts/test_obs_likelihood.py top

Repetition of Task: 100%|██████████| 100000/100000 [02:32<00:00, 657.48it/s]
Running stage 'test_obs_log_likelihood2' with command:
	python scripts/test_obs_likelihood.py top
Log likelihood of observations given top norm.: -12694.183534230084

Modifying stage 'test_obs_log_likelihood2' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.yaml dvc.lock
